In [1]:
using ITensors

In [2]:
function heisenberg_mpo(N) # Make N S=1/2 spin indices 
    sites = siteinds("S=1/2",N) 
    # Input the operator terms 
    
    os = OpSum() 
    
    for i=1:N-1 
        os += "Sz",i,"Sz",i+1 
        os += 1/2,"S+",i,"S-",i+1 
        os += 1/2,"S-",i,"S+",i+1 
    end 
    
    # Convert these terms to an MPO 
    H = MPO(os,sites)
    return H, sites
end

heisenberg_mpo (generic function with 1 method)

In [3]:
N = 5

H, sites = heisenberg_mpo(N)

println(H)
println(sites)

MPO
[1] ((dim=5|id=291|"Link,l=1"), (dim=2|id=240|"S=1/2,Site,n=1")', (dim=2|id=240|"S=1/2,Site,n=1"))
[2] ((dim=5|id=291|"Link,l=1"), (dim=5|id=247|"Link,l=2"), (dim=2|id=431|"S=1/2,Site,n=2")', (dim=2|id=431|"S=1/2,Site,n=2"))
[3] ((dim=5|id=247|"Link,l=2"), (dim=5|id=274|"Link,l=3"), (dim=2|id=880|"S=1/2,Site,n=3")', (dim=2|id=880|"S=1/2,Site,n=3"))
[4] ((dim=5|id=274|"Link,l=3"), (dim=5|id=846|"Link,l=4"), (dim=2|id=391|"S=1/2,Site,n=4")', (dim=2|id=391|"S=1/2,Site,n=4"))
[5] ((dim=5|id=846|"Link,l=4"), (dim=2|id=422|"S=1/2,Site,n=5")', (dim=2|id=422|"S=1/2,Site,n=5"))

Index{Int64}[(dim=2|id=240|"S=1/2,Site,n=1"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=2|id=880|"S=1/2,Site,n=3"), (dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=422|"S=1/2,Site,n=5")]


In [17]:
psi0 = randomMPS(sites,10)

MPS
[1] ((dim=2|id=240|"S=1/2,Site,n=1"), (dim=10|id=148|"Link,l=1"))
[2] ((dim=10|id=148|"Link,l=1"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=8|id=399|"Link,l=2"))
[3] ((dim=8|id=399|"Link,l=2"), (dim=2|id=880|"S=1/2,Site,n=3"), (dim=4|id=674|"Link,l=3"))
[4] ((dim=4|id=674|"Link,l=3"), (dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=350|"Link,l=4"))
[5] ((dim=2|id=350|"Link,l=4"), (dim=2|id=422|"S=1/2,Site,n=5"))


In [6]:
nsweeps = 5
maxdim = [10,20,100,100,200]
cutoff = [1E-10]

1-element Vector{Float64}:
 1.0e-10

In [7]:
energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-1.9278861936370253  maxlinkdim=4 maxerr=8.39E-17 time=14.106
After sweep 2 energy=-1.9278862533179897  maxlinkdim=4 maxerr=1.66E-16 time=0.007
After sweep 3 energy=-1.927886253317992  maxlinkdim=4 maxerr=1.37E-16 time=0.004
After sweep 4 energy=-1.927886253317993  maxlinkdim=4 maxerr=1.32E-16 time=0.019
After sweep 5 energy=-1.9278862533179955  maxlinkdim=4 maxerr=1.23E-16 time=0.005


(-1.9278862533179955, MPS
[1] ((dim=2|id=219|"Link,l=1"), (dim=2|id=240|"S=1/2,Site,n=1"))
[2] ((dim=4|id=834|"Link,l=2"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=2|id=219|"Link,l=1"))
[3] ((dim=2|id=880|"S=1/2,Site,n=3"), (dim=4|id=641|"Link,l=3"), (dim=4|id=834|"Link,l=2"))
[4] ((dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=938|"Link,l=4"), (dim=4|id=641|"Link,l=3"))
[5] ((dim=2|id=422|"S=1/2,Site,n=5"), (dim=2|id=938|"Link,l=4"))
)

In [8]:
S_z = [1 0; 0 -1]/2
S_plus = [0 1; 0 0]
S_minus = [0 0; 1 0]
Identity = [1 0; 0 1] #or Using LinearAlgebra and Matrix{Float64}(I, 2, 2)

println(S_z, S_plus, S_minus, Identity)

[0.5 0.0; 0.0 -0.5][0 1; 0 0][0 0; 1 0][1 0; 0 1]


In [9]:
using LinearAlgebra

In [10]:
function SZ_Term(N)
    Matrix_Sz = zeros(2^N, 2^N)
    
    for i=1:N-1
        M = Identity
        
        if i == 1
            M = S_z
        end

        for j=2:N 
            if j == i || j == i + 1
            M = kron(M, S_z)
            else
            M = kron(M, Identity)        
            end
        end 

        Matrix_Sz = Matrix_Sz + M
    end

    return Matrix_Sz
end

function SPlus_Minus_Term(N)
    Matriz_SP_SM = zeros(2^N, 2^N)
    
    for i=1:N-1
        M = Identity
        
        if i == 1
            M = S_plus
        end

        M = M/2

        for j=2:N 
            if j == i
            M = kron(M, S_plus)
            elseif j == i + 1
            M = kron(M, S_minus)
            else
            M = kron(M, Identity)        
            end
        end 

        Matriz_SP_SM = Matriz_SP_SM + M
    end

    return Matriz_SP_SM
end

function SMinus_Plus_Term(N)
    Matriz_SM_SP = zeros(2^N, 2^N)
    
    for i=1:N-1
        M = Identity
        
        if i == 1
            M = S_minus
        end

        M = M/2

        for j=2:N 
            if j == i
            M = kron(M, S_minus)
            elseif j == i + 1
            M = kron(M, S_plus)
            else
            M = kron(M, Identity)        
            end
        end 

        Matriz_SM_SP = Matriz_SM_SP + M
    end

    return Matriz_SM_SP
end

function Exact_Diagonalization(N)

    H = SZ_Term(N) + SPlus_Minus_Term(N) + SMinus_Plus_Term(N)
    return eigen(H).values, eigen(H).vectors   
end

Exact_Diagonalization (generic function with 1 method)

In [11]:
# using Kronecker
# kronecker(I, I, I,I)
# I⊗I⊗I⊗I

In [12]:
SZ_Term(3) == kron(S_z, S_z, Identity) + kron(Identity, S_z, S_z)
SPlus_Minus_Term(3) == kron(S_plus, S_minus, Identity)/2 + kron(Identity, S_plus, S_minus)/2
SMinus_Plus_Term(3) == kron(S_minus, S_plus, Identity)/2 + kron(Identity, S_minus, S_plus)/2

true

In [13]:
values, vectors = Exact_Diagonalization(5)

([-1.9278862533179946, -1.9278862533179932, -1.207106781186551, -1.2071067811865437, -0.8090169943749478, -0.8090169943749477, -0.8090169943749469, -0.8090169943749452, -0.6608185871316489, -0.6608185871316484  …  0.8090169943749472, 0.8090169943749475, 0.8090169943749477, 0.8090169943749481, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0000000000000002], [0.0 0.0 … 0.0 0.0; -3.40136667881805e-21 3.7269449679189215e-19 … 0.0 0.23863476681970214; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0])

In [14]:
values

32-element Vector{Float64}:
 -1.9278862533179946
 -1.9278862533179932
 -1.207106781186551
 -1.2071067811865437
 -0.8090169943749478
 -0.8090169943749477
 -0.8090169943749469
 -0.8090169943749452
 -0.6608185871316489
 -0.6608185871316484
 -0.309016994374949
 -0.3090169943749465
 -0.3090169943749461
  ⋮
  0.588704840449642
  0.5887048404496427
  0.8090169943749472
  0.8090169943749475
  0.8090169943749477
  0.8090169943749481
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0000000000000002

In [16]:
energy, psi_0 = dmrg(H,psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-1.9278861936370253  maxlinkdim=4 maxerr=8.39E-17 time=0.016
After sweep 2 energy=-1.9278862533179897  maxlinkdim=4 maxerr=1.66E-16 time=0.025
After sweep 3 energy=-1.927886253317992  maxlinkdim=4 maxerr=1.37E-16 time=0.038
After sweep 4 energy=-1.927886253317993  maxlinkdim=4 maxerr=1.32E-16 time=0.009
After sweep 5 energy=-1.9278862533179955  maxlinkdim=4 maxerr=1.23E-16 time=0.006


(-1.9278862533179955, MPS
[1] ((dim=2|id=370|"Link,l=1"), (dim=2|id=240|"S=1/2,Site,n=1"))
[2] ((dim=4|id=729|"Link,l=2"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=2|id=370|"Link,l=1"))
[3] ((dim=2|id=880|"S=1/2,Site,n=3"), (dim=4|id=988|"Link,l=3"), (dim=4|id=729|"Link,l=2"))
[4] ((dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=197|"Link,l=4"), (dim=4|id=988|"Link,l=3"))
[5] ((dim=2|id=422|"S=1/2,Site,n=5"), (dim=2|id=197|"Link,l=4"))
)

In [18]:
energy, psi_1 = dmrg(H,[psi_0], psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-1.927885002240556  maxlinkdim=4 maxerr=1.24E-15 time=0.392
After sweep 2 energy=-1.9278862533176033  maxlinkdim=4 maxerr=3.33E-17 time=0.004
After sweep 3 energy=-1.9278862533179948  maxlinkdim=4 maxerr=1.91E-16 time=0.005
After sweep 4 energy=-1.927886253317995  maxlinkdim=4 maxerr=2.03E-16 time=0.031
After sweep 5 energy=-1.9278862533179952  maxlinkdim=4 maxerr=1.57E-16 time=0.005


(-1.9278862533179952, MPS
[1] ((dim=2|id=441|"Link,l=1"), (dim=2|id=240|"S=1/2,Site,n=1"))
[2] ((dim=4|id=491|"Link,l=2"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=2|id=441|"Link,l=1"))
[3] ((dim=2|id=880|"S=1/2,Site,n=3"), (dim=4|id=92|"Link,l=3"), (dim=4|id=491|"Link,l=2"))
[4] ((dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=100|"Link,l=4"), (dim=4|id=92|"Link,l=3"))
[5] ((dim=2|id=422|"S=1/2,Site,n=5"), (dim=2|id=100|"Link,l=4"))
)

In [19]:
energy, psi_2 = dmrg(H,[psi_0, psi_1], psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-1.2071050294411456  maxlinkdim=4 maxerr=4.63E-16 time=0.014
After sweep 2 energy=-1.2071067811527736  maxlinkdim=4 maxerr=1.66E-16 time=0.031
After sweep 3 energy=-1.2071067811865475  maxlinkdim=4 maxerr=8.54E-17 time=0.040
After sweep 4 energy=-1.2071067811865472  maxlinkdim=4 maxerr=1.30E-16 time=0.016
After sweep 5 energy=-1.207106781186548  maxlinkdim=4 maxerr=1.65E-16 time=0.012


(-1.207106781186548, MPS
[1] ((dim=2|id=402|"Link,l=1"), (dim=2|id=240|"S=1/2,Site,n=1"))
[2] ((dim=4|id=641|"Link,l=2"), (dim=2|id=431|"S=1/2,Site,n=2"), (dim=2|id=402|"Link,l=1"))
[3] ((dim=2|id=880|"S=1/2,Site,n=3"), (dim=4|id=758|"Link,l=3"), (dim=4|id=641|"Link,l=2"))
[4] ((dim=2|id=391|"S=1/2,Site,n=4"), (dim=2|id=959|"Link,l=4"), (dim=4|id=758|"Link,l=3"))
[5] ((dim=2|id=422|"S=1/2,Site,n=5"), (dim=2|id=959|"Link,l=4"))
)